In [14]:
import pandas as pd
from sklearn.model_selection import train_test_split
import sklearn.metrics as metric
from math import sqrt
from prettytable import PrettyTable as PT

In [2]:
from surprise import KNNWithMeans
from surprise.model_selection import GridSearchCV
from surprise import Dataset
from surprise import accuracy
from surprise import Reader


In [3]:
#pip install scikit-surprise
#!pip install PrettyTable


In [3]:
df = pd.read_json('Movies_and_TV.json', lines = True)

In [4]:
df.head(5)

,overall,verified,reviewTime,reviewerID,asin,style,reviewerName,reviewText,summary,unixReviewTime,vote,image
0,5,True,"11 9, 2012",A2M1CU2IRZG0K9,0005089549,{'Format:': ' VHS Tape'},Terri,So sorry I didn't purchase this years ago when...,Amazing!,1352419200,NaN,NaN
1,5,True,"12 30, 2011",AFTUJYISOFHY6,0005089549,{'Format:': ' VHS Tape'},Melissa D. Abercrombie,Believe me when I tell you that you will recei...,Great Gospel VHS of the Cathedrals!,1325203200,NaN,NaN
2,5,True,"04 21, 2005",A3JVF9Y53BEOGC,000503860X,{'Format:': ' DVD'},Anthony Thompson,"I have seen X live many times, both in the ear...",A great document of a great band,1114041600,11.0,NaN
3,5,True,"04 6, 2005",A12VPEOEZS1KTC,000503860X,{'Format:': ' DVD'},JadeRain,"I was so excited for this! Finally, a live co...",YES!! X LIVE!!,1112745600,5.0,NaN
4,5,True,"12 3, 2010",ATLZNVLYKP9AZ,000503860X,{'Format:': ' DVD'},T. Fisher,X is one of the best punk bands ever. I don't ...,X have still got it,1291334400,5.0,NaN


In [14]:
df.describe(include='all')

,overall,verified,reviewTime,reviewerID,asin,style,reviewerName,reviewText,summary,unixReviewTime,vote,image
count,1.584082e+06,1584082,1584082,1584082,1584082,1310814,1584001,1582629,1583547,1.584082e+06,124722,6591
unique,NaN,2,7272,840372,456992,41,598429,1254879,713973,NaN,218,6402
top,NaN,True,"02 20, 2015",A3W4D8XOGLWUN5,B0135P6PZA,{'Format:': ' MP3 Music'},Amazon Customer,Great song,Five Stars,NaN,2,[https://images-na.ssl-images-amazon.com/image...
freq,NaN,1217667,5038,1326,3411,993745,82289,7127,399073,NaN,50192,11
mean,4.660555e+00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.408211e+09,NaN,NaN
std,8.440314e-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.857646e+07,NaN,NaN
min,1.000000e+00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.773056e+08,NaN,NaN
25%,5.000000e+00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.374883e+09,NaN,NaN
50%,5.000000e+00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.420070e+09,NaN,NaN
75%,5.000000e+00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.457222e+09,NaN,NaN


In [5]:
null_data = pd.DataFrame(df.isnull().mean() * 100)
null_data

,0
overall,0.000000
verified,0.000000
reviewTime,0.000000
reviewerID,0.000000
asin,0.000000
style,0.749532
reviewerName,0.000000
reviewText,0.049969
summary,0.006246
unixReviewTime,0.000000


In [6]:
df.reviewText.isnull().value_counts()

False    16002
True         8
Name: reviewText, dtype: int64

In [7]:
new_df = df[df.reviewText.notna()]

In [8]:
new_df.shape

(16002, 12)

In [9]:
after_df = new_df[['overall', 'unixReviewTime', 'reviewerID', 'asin','reviewText']]
after_df.rename(columns={'overall': 'ratings'}, inplace=True)

C:\Users\woshi\anaconda3\lib\site-packages\pandas\core\frame.py:5039: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


In [10]:
after_df.head()

,ratings,unixReviewTime,reviewerID,asin,reviewText
0,5,1352419200,A2M1CU2IRZG0K9,0005089549,So sorry I didn't purchase this years ago when...
1,5,1325203200,AFTUJYISOFHY6,0005089549,Believe me when I tell you that you will recei...
2,5,1114041600,A3JVF9Y53BEOGC,000503860X,"I have seen X live many times, both in the ear..."
3,5,1112745600,A12VPEOEZS1KTC,000503860X,"I was so excited for this! Finally, a live co..."
4,5,1291334400,ATLZNVLYKP9AZ,000503860X,X is one of the best punk bands ever. I don't ...


In [11]:
after_df.drop('unixReviewTime', axis=1, inplace=True)

C:\Users\woshi\anaconda3\lib\site-packages\pandas\core\frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [12]:
after_df.head()

,ratings,reviewerID,asin,reviewText
0,5,A2M1CU2IRZG0K9,0005089549,So sorry I didn't purchase this years ago when...
1,5,AFTUJYISOFHY6,0005089549,Believe me when I tell you that you will recei...
2,5,A3JVF9Y53BEOGC,000503860X,"I have seen X live many times, both in the ear..."
3,5,A12VPEOEZS1KTC,000503860X,"I was so excited for this! Finally, a live co..."
4,5,ATLZNVLYKP9AZ,000503860X,X is one of the best punk bands ever. I don't ...


In [15]:
training_data, testing_data = train_test_split(
    after_df, test_size = 0.25, random_state = 42
)

In [16]:
reviewer = after_df.groupby('reviewerID').count()
high_Rate_Reviewer = reviewer[reviewer.ratings >= 3].index
top_reviews_Df = after_df[after_df.reviewerID.isin(high_Rate_Reviewer)]

In [17]:
asin = after_df.groupby('asin').count()
high_Rate_Asins = asin[asin.ratings >= 3].index
high_Asin = top_reviews_Df[top_reviews_Df.asin.isin(high_Rate_Asins)]


In [18]:
high_Asin.head()

,ratings,reviewerID,asin,reviewText
23,5,A1E7VTRDMI4XMV,0005092663,Really excellent piece of work showing the bel...
25,5,AZB4CQ9JZSUQB,0005019281,"When this first aired in 1979, I enjoyed it so..."
39,3,A101IGU6UDKW3X,0005019281,Well made Christmas movie. It's a little slow ...
214,5,A23M34LR1SQHS9,0005019281,One of the better remakes of A Christmas Carol...
254,5,A1GHUN5HXMHZ89,0005019281,"The first time I saw ""An American Christmas Ca..."


In [19]:
from surprise.model_selection import train_test_split
# converting data to surprise reader format
surprise_reader = Reader(rating_scale = (0.1, 5.0))
surprise_data = Dataset.load_from_df(
    high_Asin[['reviewerID', 'asin', 'ratings']],
    surprise_reader)
# Training the Model
training_set, testing_set = train_test_split(surprise_data, 
                                             test_size=0.25,
                                             random_state=42)


training_set.__class__, testing_set.__class__

(surprise.trainset.Trainset, list)

In [23]:
knn_model_10 = KNNWithMeans(k=10, min_k=6, 
                         sim_options={'name': 'pearson_baseline', 'user_based': True})
knn_model_10.fit(training_set)

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.


In [21]:
knn_model_15 = KNNWithMeans(k=15, min_k=6, 
                         sim_options={'name': 'pearson_baseline', 'user_based': True})
knn_model_15.fit(training_set)

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.


In [22]:
knn_model_20 = KNNWithMeans(k=20, min_k=6, 
                         sim_options={'name': 'pearson_baseline', 'user_based': True})
knn_model_20.fit(training_set)

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.


In [24]:
eval_table = PT()
# print(eval_table)
eval_table.field_names = ["Model Type", "RMSE Error"]

In [25]:
knn_10_test_predictions = knn_model_10.test(testing_set)
knn_15_test_predictions = knn_model_15.test(testing_set)
knn_20_test_predictions = knn_model_20.test(testing_set)
eval_table.add_row(["KNN k=10", accuracy.rmse(knn_10_test_predictions)])
eval_table.add_row(["KNN k=15", accuracy.rmse(knn_15_test_predictions)])
eval_table.add_row(["KNN k=20", accuracy.rmse(knn_20_test_predictions)])

RMSE: 1.1332
RMSE: 1.1333
RMSE: 1.1334


In [26]:
from surprise import KNNBasic, SVD, NormalPredictor, KNNBaseline,KNNWithMeans, KNNWithZScore, BaselineOnly, CoClustering, Reader, dataset, accuracy
from collections import defaultdict
svd_model_50 = SVD(n_factors=50,reg_all=0.02)
svd_model_50.fit(training_set)

In [27]:
svd_model_25 = SVD(n_factors=25,reg_all=0.02)
svd_model_25.fit(training_set)

In [28]:
svd_model_75 = SVD(n_factors=75,reg_all=0.02)
svd_model_75.fit(training_set)

In [29]:
svd_25_test_predictions = svd_model_25.test(testing_set)
svd_50_test_predictions = svd_model_50.test(testing_set)
svd_75_test_predictions = svd_model_75.test(testing_set)

In [30]:
eval_table.add_row(["SVD n_factors=25", accuracy.rmse(svd_25_test_predictions)])
eval_table.add_row(["SVD n_factors=50", accuracy.rmse(svd_50_test_predictions)])
eval_table.add_row(["SVD n_factors=75", accuracy.rmse(svd_75_test_predictions)])

RMSE: 0.9958
RMSE: 0.9992
RMSE: 0.9939


In [31]:
print(eval_table)

+------------------+--------------------+
|    Model Type    |     RMSE Error     |
+------------------+--------------------+
|     KNN k=10     | 1.1332332967126661 |
|     KNN k=15     | 1.1333268334178086 |
|     KNN k=20     | 1.1333991058315804 |
| SVD n_factors=25 | 0.9958015976975171 |
| SVD n_factors=50 | 0.9991661498394014 |
| SVD n_factors=75 | 0.9938647075787407 |
+------------------+--------------------+


In [32]:
svd_param_grid = {
    'n_factors' : list(range(5, 100, 10)),
    'reg_all': [0.01, 0.02, 0.03]
}

In [33]:
svd_grid_search = GridSearchCV(
    SVD,
    svd_param_grid,
    measures = ['rmse'],
    cv = 5,
    refit = True
)

In [34]:
svd_grid_search.fit(surprise_data)
svd_grid_search.best_params

{'rmse': {'n_factors': 95, 'reg_all': 0.01}}

In [36]:
svd_cv_testing = svd_grid_search.test(testing_set)

In [37]:
eval_table.add_row(["Best SVD n_factors=95", accuracy.rmse(svd_cv_testing)])
print(eval_table)

RMSE: 0.7002
+-----------------------+--------------------+
|       Model Type      |     RMSE Error     |
+-----------------------+--------------------+
|        KNN k=10       | 1.1332332967126661 |
|        KNN k=15       | 1.1333268334178086 |
|        KNN k=20       | 1.1333991058315804 |
|    SVD n_factors=25   | 0.9958015976975171 |
|    SVD n_factors=50   | 0.9991661498394014 |
|    SVD n_factors=75   | 0.9938647075787407 |
| Best SVD n_factors=95 | 0.7001759133924601 |
+-----------------------+--------------------+
